In [ ]:
import os
import sys
import time
import joblib
import librosa
import random
import numpy as np
np.random.seed(15)
import pandas as pd
import scipy.io.wavfile as wav
from numpy.lib import stride_tricks

%tensorflow_version 1.x
import tensorflow as tf

from keras import backend as K
K.common.set_image_dim_ordering('th')

# Silent removal and Speaker diarization

In [ ]:
from SilentRemoval_SpeakerDiarization import remove_silence

# directory containing raw wav files
dir_name = 'dataset'

# directory where a participant folder will be created containing their
# segmented wav file
out_dir = 'processed_dataset'

# iterate through wav files in dir_name and create a segmented wav_file
for file in os.listdir(dir_name):
  if file.endswith('.wav'):
    filename = os.path.join(dir_name, file)
    remove_silence(filename, out_dir)

# Augment depressed audio and generate new samples


In [ ]:
df_train = pd.read_csv('train_split_Depression_AVEC2017.csv')
df_validate = pd.read_csv('dev_split_Depression_AVEC2017.csv')
df_dev = pd.concat([df_train, df_validate], axis=0)

for i in os.listdir('processed_dataset'):
  id = int(i[1:])
  if id in df_dev['Participant_ID'].values:
    if df_dev.loc[df_dev['Participant_ID'] == id]['PHQ8_Binary'].item() == 1:
      y, sr = librosa.load('processed_dataset/'+i+'/'+i+'_no_silence.wav')  
      y_changed = librosa.effects.time_stretch(y, rate=1.07)
      librosa.output.write_wav('augmented_dataset/'+i+'_no_silence.wav' ,y_changed, sr)


# Feature Extraction from train dataset

From original preprocessed wav files

In [ ]:
from FeatureExtraction import stft_matrix
from FeatureExtraction import get_random_samples

depressed_data = []
normal_data = []

base = 'processed_dataset/'

for i in os.listdir(base):
  if int(i[1:]) in set(df_dev['Participant_ID'].values):
    try:
      data = get_random_samples(stft_matrix(base+i+'/'+i+'_no_silence.wav'),46,125)
      if (df_dev.loc[df_dev['Participant_ID'] == int(i[1:])]['PHQ8_Binary'].item()==0):
        normal_data = normal_data + data
      else:
        depressed_data = depressed_data + data
    except:
      print(i)

From augmented depressed wav files

In [ ]:
base = 'augmented_dataset/'
for i in os.listdir(base):
  x=int(i[1:4])
  if x in set(df_dev['Participant_ID'].values):
    try:
      data = get_random_samples(stft_matrix(base+i),46,125)
      depressed_data = depressed_data + data
    except:
      print(i)

print(np.array(depressed_data).shape)
print(np.array(normal_data).shape)

Seprate train, validation datasets for CNN and GSOM models

In [ ]:
random.shuffle(depressed_data)
random.shuffle(depressed_data)

cnn_train_data = depressed_data[:1702]+ normal_data[:1802]
cnn_validation_data = depressed_data[1702:3202]+ normal_data[1802:3402]
gsom_validation_data = depressed_data[3202:]+ normal_data[3402:]

In [ ]:
cnn_train_labels = []
cnn_validation_labels = []
gsom_validation_labels = []

for i in range(1702):
  cnn_train_labels.append(1)
for i in range(1802):
  cnn_train_labels.append(0)

for i in range(1500):
  cnn_validation_labels.append(1)
for i in range(1600):
  cnn_validation_labels.append(0)

for i in range(202):
  gsom_validation_labels.append(1)
for i in range(232):
  gsom_validation_labels.append(0)

In [ ]:
joblib.dump(np.array(cnn_train_data),'cnn_train_data.joblib')
joblib.dump(np.array(cnn_validation_data),'cnn_validation_data.joblib')
joblib.dump(np.array(gsom_validation_data),'gsom_validation_data.joblib')

# CNN model creation

In [ ]:
from FeaturePreprocess import prep_train_test
from FeaturePreprocess import keras_img_prep
from CNNmodel import cnn
from Evaluation import cnn_model_performance

CNN model configs

In [ ]:
batch_size = 32
nb_classes = 2
epochs = 1

preprocess train data

In [ ]:
# normalalize data and prep for Keras
X_train, X_test, y_train, y_test = prep_train_test(cnn_train_data, np.array(cnn_train_labels),cnn_validation_data, np.array(cnn_validation_labels),nb_classes=nb_classes)

# 513x125x1 for spectrogram with crop size of 125 pixels
img_rows, img_cols, img_depth = X_train.shape[1], X_train.shape[2], 1

# reshape image input for Keras
# used Theano dim_ordering (th), (# chans, # images, # rows, # cols)
X_train, X_test, input_shape = keras_img_prep(X_train, X_test, img_depth,img_rows, img_cols)

Fit the cnn model


In [ ]:
model, history = cnn(X_train, y_train, X_test, y_test, batch_size,nb_classes, epochs, input_shape)

Evaluate the model

In [ ]:
y_train_pred, y_test_pred, y_train_pred_proba, y_test_pred_proba, conf_matrix = cnn_model_performance(model, X_train, X_test, y_train, y_test)

# custom evaluation metrics
print('Calculating additional test metrics...')
accuracy = float(conf_matrix[0][0] + conf_matrix[1][1]) / np.sum(conf_matrix)
precision = float(conf_matrix[0][0]) / (conf_matrix[0][0] + conf_matrix[0][1])
recall = float(conf_matrix[0][0]) / (conf_matrix[0][0] + conf_matrix[1][0])
f1_score = 2 * (precision * recall) / (precision + recall)
print("Accuracy: {}".format(accuracy))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("F1-Score: {}".format(f1_score))

# GSOM model creation


In [ ]:
from FeaturePreprocess import prep_full_test

sys.path.append('GSOM')

import data_parser as Parser
from util import utilities as Utils
from util import display as Display_Utils
from params import params as Params
from core4 import core_controller as Core

CNN validation data output to train GSOM

In [ ]:
cnn_validation_data = joblib.load('cnn_validation_data.joblib')

gsom_train_data = prep_full_test(cnn_validation_data)

# 513x125x1 for spectrogram with crop size of 125 pixels
img_rows, img_cols, img_depth = gsom_train_data.shape[1], gsom_train_data.shape[2], 1

# reshape image input for Keras
# used Theano dim_ordering (th), (# chans, # images, # rows, # cols)
gsom_train_data, input_shape = keras_img_prep(gsom_train_data, img_depth, img_rows, img_cols)

cnnPredictions=model.predict(gsom_train_data)
cnnPredictions.shape

In [ ]:
def array_to_comma_separated(array):
    return ','.join([str(i) for i in array]) 

f=open("GSOM/data/cnn_5100_input_file_to_gsom","w")       
for i in range(len(cnnPredictions)):
  f.write(str(i)+","+array_to_comma_separated(cnnPredictions[i])+","+str(cnn_validation_labels[i])+"\n")  

f=open("GSOM/data/cnn_5100_input_file_to_gsom")
len(f.readlines())

GSOM configs


In [ ]:
SF = 0.7
forget_threshold = 80  # To include forgetting, threshold should be < learning iterations.
temporal_contexts = 1  # If stationary data - keep this at 1
learning_itr = 100
smoothing_irt = 50
plot_for_itr = 4  # Unused parameter - just for visualization. Keep this as it is.

# File Config
data_filename = "GSOM/data/cnn_5100_input_file_to_gsom"
output_save_location = 'GSOM/output/'

GSOM model

In [ ]:
from GSOMmodel import GSOM_model

GSOM_model(SF,forget_threshold,temporal_contexts,learning_itr,smoothing_irt,plot_for_itr,data_filename,output_save_location,"5100_for_0_7")

# Evaluate CNN+GSOM Combined Model


Load CNN model

In [ ]:
from keras.models import load_model
model = load_model('cnn_5100.h5')

model.pop()
model.pop()
model.pop()
model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
model.summary()

Load validation data

In [ ]:
gsom_validation_data = joblib.load('gsom_validation_data.joblib')

gsom_validation_data = prep_full_test(gsom_validation_data)

# 513x125x1 for spectrogram with crop size of 125 pixels
img_rows, img_cols, img_depth = gsom_validation_data.shape[1], gsom_validation_data.shape[2], 1

# reshape image input for Keras
# used Theano dim_ordering (th), (# chans, # images, # rows, # cols)
gsom_validation_data, input_shape = keras_img_prep(gsom_validation_data, img_depth, img_rows, img_cols)

cnnPredictions=model.predict(gsom_validation_data)
cnnPredictions.shape

In [ ]:
f=open("GSOM/data/cnn_5100_validate_file_to_gsom","w")
for i in range(len(cnnPredictions)):
  f.write(str(i)+","+array_to_comma_separated(cnnPredictions[i])+","+str(gsom_validation_labels[i])+"\n") 

f=open("GSOM/data/cnn_5100_validate_file_to_gsom")
len(f.readlines()) 

Load test data

In [ ]:
X_full_test = joblib.load('X_full_test_saved.joblib')
cnnPredictionsForGsomTest = model.predict(X_full_test)
 
f=open("GSOM/data/cnn_5100_test_file_to_gsom","w")
for i in range(len(cnnPredictionsForGsomTest)):
  f.write(str(i)+","+array_to_comma_separated(cnnPredictionsForGsomTest[i])+"\n")

f=open("GSOM/data/cnn_5100_test_file_to_gsom")
print(len(f.readlines()))

Node labeling algorithm

In [ ]:
import data_parser as Parser

input_vector_database, labels, classes = Parser.InputParser.parse_input_train_data('GSOM/data/cnn_5100_input_file_to_gsom', None)
input_vector_database_validate, validate_labels = Parser.InputParser.parse_input_test_data('GSOM/data/cnn_5100_validate_file_to_gsom', None)
input_vector_database_test, test_labels = Parser.InputParser.parse_input_test_data('GSOM/data/cnn_5100_test_file_to_gsom', None)

def get_labels_in_radius(gsom_nodemap,radius,x,y):  
  label_list = get_winner_labels(gsom_nodemap,x+radius,y)+get_winner_labels(gsom_nodemap,x-radius,y)+get_winner_labels(gsom_nodemap,x,y+radius)+get_winner_labels(gsom_nodemap,x,y-radius)+get_winner_labels(gsom_nodemap,x+radius,y+radius)+get_winner_labels(gsom_nodemap,x+radius,y-radius)+get_winner_labels(gsom_nodemap,x-radius,y+radius)+get_winner_labels(gsom_nodemap,x-radius,y-radius)
  return label_list

def get_winner_labels(gsom_nodemap,n,m):
  winner_key = Utils.Utilities.generate_index(n, m)
  try:
    mapped_input_labels=gsom_nodemap[winner_key].get_mapped_labels()
    return [str(classes[lbl_id]) for lbl_id in mapped_input_labels]

  except KeyError:
    #if the new generated key does not exist in the original key map
    return []

Evaluate gsom on validation data

In [ ]:
def get_predictions_from_gsom(gsom_nodemap,threshold):
  test_predictions=[]

  for test_input_id in validate_labels:
    #print('test_input_id: ',test_input_id)
    winner=Utils.Utilities.select_winner(gsom_nodemap, input_vector_database_validate[0][test_input_id], Params.DistanceFunction.EUCLIDEAN, -1)
    label_list = get_winner_labels(gsom_nodemap,winner.x,winner.y)

    radius=0
    x = winner.x
    y = winner.y

    while(len(label_list)==0):      
      radius=radius+1
      label_list=label_list+get_labels_in_radius(gsom_nodemap,radius,x,y)
      
    maxCountElement = max(label_list,key=label_list.count)

    a=label_list.count('1')
    b=label_list.count('0')
    p=a/(a+b)

    if p>=threshold:
      test_predictions.append(1)
    else:
      test_predictions.append(0)

  print('test_predictions ',test_predictions)
  print('num test_predictions: ',len(test_predictions))
  return test_predictions

Evaluate combined model on test data

In [ ]:
def get_predictions_from_gsom_test(gsom_nodemap,threshold):
  test_predictions=[]

  for test_input_id in test_labels:
    #print('test_input_id: ',test_input_id)
    winner=Utils.Utilities.select_winner(gsom_nodemap, input_vector_database_test[0][test_input_id], Params.DistanceFunction.EUCLIDEAN, -1)
    label_list = get_winner_labels(gsom_nodemap,winner.x,winner.y)

    radius=0
    x = winner.x
    y = winner.y

    while(len(label_list)==0):      
      radius=radius+1
      label_list=label_list+get_labels_in_radius(gsom_nodemap,radius,x,y)
    
    a=label_list.count('1')
    b=label_list.count('0')
    p=a/(a+b)

    #maxCountElement = max(label_list,key=label_list.count)
    if p>=threshold:
      test_predictions.append(1)
    else:
      test_predictions.append(0)

  print('test_predictions ',test_predictions)
  print('num test_predictions: ',len(test_predictions))

  bundled_test_predictions=[]
  for i in range(0,37):
    start_idx=46*i
    end_idx=start_idx+46
    bundled_test_predictions.append(int(np.round(np.average(test_predictions[start_idx:end_idx]))))
    #print(bundled_test_predictions)
  np.asarray(bundled_test_predictions)
  print('bundled_test_predictions ',bundled_test_predictions)
  print('num bundled_test_predictions:',len(bundled_test_predictions))

  return bundled_test_predictions,test_predictions

Summary of final results

In [ ]:
from Evaluation import evaluate_accuracies, overall_evaluation_test, overall_evaluation_unbundle 

threshold = 0.35

gsom_nodemap_5000_for_0_5 = joblib.load('GSOM/output/gsom_nodemap_5100_for_0_7.joblib')
prediction_validate = get_predictions_from_gsom(gsom_nodemap_5000_for_0_5,threshold)
evaluate_accuracies(prediction_validate)
print("****************************************")
predictions,test_predictions = get_predictions_from_gsom_test(gsom_nodemap_5000_for_0_5,threshold)
overall_evaluation_test(predictions)
overall_evaluation_unbundle(test_predictions)